In [1]:
import wikipediaapi
wiki = wikipediaapi.Wikipedia('en')

## Try Biomedical Category First

In [2]:
def print_categorymembers(categorymembers, level=0, max_level=2):
        for c in categorymembers.values():
            print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
            if c.ns == wikipediaapi.Namespace.CATEGORY and level <= max_level:
                print_categorymembers(c.categorymembers, level + 1)


cat = wiki.page("Category:Biomedical")
print("Category members: Category:Biomedical")
print_categorymembers(cat.categorymembers)

Category members: Category:Biomedical


## Find Proper Category for Biomedical Field
### From biomedical engineering introduction page

In [3]:
be = wiki.page('Biomedical engineering')
be.exists()

True

In [4]:
be = wiki.page('Biomedical sciences')
be.exists()

True

In [5]:
be = wiki.page('Biomedicine')
be.exists()

True

In [6]:
be = wiki.page('Biomedical')

def print_categories(page):
        categories = page.categories
        for title in sorted(categories.keys()):
            print(title, categories[title])


print("Categories")
print_categories(be)

Categories
Category:Biomedicine Category:Biomedicine (id: ??, ns: 14)
Category:Branches of biology Category:Branches of biology (id: ??, ns: 14)
Category:Pages using Columns-list with deprecated parameters Category:Pages using Columns-list with deprecated parameters (id: ??, ns: 14)
Category:Pages using div col without cols and colwidth parameters Category:Pages using div col without cols and colwidth parameters (id: ??, ns: 14)
Category:Veterinary medicine Category:Veterinary medicine (id: ??, ns: 14)
Category:Webarchive template wayback links Category:Webarchive template wayback links (id: ??, ns: 14)


### Find all articles in Biomedical engineering

In [7]:
def print_categorymembers(categorymembers, level=0, max_level=2):
        for c in categorymembers.values():
            print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
            if c.ns == wikipediaapi.Namespace.CATEGORY and level <= max_level:
                print_categorymembers(c.categorymembers, level + 1)


cat = wiki.page("Category:Biomedical engineering")
print("Category members: Category:Biomedical engineering")
#print_categorymembers(cat.categorymembers)

Category members: Category:Biomedical engineering


### Add relationship by the links between pages

In [8]:
be = wiki.page('Biomedical')

def print_links(page):
        links = page.links
        for title in sorted(links.keys()):
            print("%s: %s" % (title, links[title]))

print_links(be)

Addiction medicine: Addiction medicine (id: ??, ns: 0)
Adenosine triphosphate: Adenosine triphosphate (id: ??, ns: 0)
Adolescent medicine: Adolescent medicine (id: ??, ns: 0)
Allergy: Allergy (id: ??, ns: 0)
Allied health: Allied health (id: ??, ns: 0)
Anatomical pathology: Anatomical pathology (id: ??, ns: 0)
Anatomy: Anatomy (id: ??, ns: 0)
Andrology: Andrology (id: ??, ns: 0)
Anesthesiology: Anesthesiology (id: ??, ns: 0)
Angiology: Angiology (id: ??, ns: 0)
Bachelor of Medical Sciences: Bachelor of Medical Sciences (id: ??, ns: 0)
Bachelor of Medicine, Bachelor of Surgery: Bachelor of Medicine, Bachelor of Surgery (id: ??, ns: 0)
BioMedicines (company): BioMedicines (company) (id: ??, ns: 0)
Biochemistry: Biochemistry (id: ??, ns: 0)
Bioinformatics: Bioinformatics (id: ??, ns: 0)
Biological engineering: Biological engineering (id: ??, ns: 0)
Biology: Biology (id: ??, ns: 0)
Biomedical sciences: Biomedical sciences (id: ??, ns: 0)
Biostatistics: Biostatistics (id: ??, ns: 0)
Biotech

In [9]:
test = wiki.page('Biomedical Equipment Technician')
test.title

'Biomedical Equipment Technician'

## Interface with mongoDB

In [10]:
from pymongo import MongoClient
import json

config = json.load(open("config.json"))

client = MongoClient()
client = MongoClient(config["server"], 27017)
db = client.Biomedical

In [11]:
# Search all categories that is a member of main category: Biochemical
def get_allsubcategories(cat, level = 0, max_level = 8):
    for c in cat.categorymembers.values():
        if c.ns == wikipediaapi.Namespace.CATEGORY and level <= max_level:
            if db.categories.find({'title':c.title}).count() == 0:
                db.categories.insert_one({'level':level+1, 'type':'sub_category','title':c.title,'parent':cat.title})
                get_allsubcategories(c,level+1)
            else:
                minlevel = min(level+1,db.categories.find_one({'title':c.title})['level'])
                db.categories.update_one({'title':c.title},{'$set':{'level':minlevel}})

In [12]:
names = ["Category:Biomedical","Category:Biomedical engineering","Category:Biomedical sicences","Category:Biomedicine"]
for name in names:
    cat = wiki.page(name)
    db.categories.insert_one({'level':0, 'type':'main_category','title':name})
    get_allsubcategories(cat)

In [ ]:
#For all categories, get their pages
def get_allrelatednodes():
    for cat in db.categories.find({},no_cursor_timeout=True):
        pages = wiki.page(cat['title']).categorymembers.values()
        for page in pages:
            if page.ns != wikipediaapi.Namespace.CATEGORY:
                if db.pages.find({'title':page.title}).count() == 0:
                    links = []
                    try:
                        for link in page.links.keys():
                            links.append(link)
                        db.pages.insert_one({'title':page.title,'category':[cat['title']],'links':links})
                    except:
                        print("No links for "+page.title)
                else:
                    newcat = db.pages.find_one({'title':page.title})['category']
                    if cat['title'] not in newcat:
                        newcat.append(cat['title'])
                    db.pages.update_one({'title':page.title},{'$set':{'category':newcat}})
                    
get_allrelatednodes()

No links for Ultimate Comics: Avengers
No links for Ultimate Comics: The Ultimates


In [18]:
pages = wiki.page("Category:Biomedical scientist").categorymembers.values()
for page in pages:
    print(page)

In [20]:
#Link the pages and categories together
def get_alledges():
    get_allcatedges()
    get_allpageedges()
    
def get_allcatedges():
    cats = db.categories.find({})
    for cat in cats:
        if cat['level'] > 0:
            db.cat_edges.insert_one({'Parent':db.categories.find_one({'title':cat['parent']}),'Child':cat,'content':'added via'})

def get_allpageedges():
    pages = db.pages.find({})
    for page in pages:
        for linktitle in page['links']:
            linkpage = db.pages.find_one({'title':linktitle})
            if db.page_edges.find({'Source':page,'Link':linkpage}).count() == 0:
                db.page_edges.insert_one({'Source':page,'Link':linkpage,'content':'has link to'})
            else:
                db.page_edges.update_one({'Source':page,'Link':linkpage},{'$set':{'content':'linked with each other'}})

#get_alledges()

In [71]:
db.categories.find_one({'title':"Category:Biomedical engineering in fiction"})

{'_id': ObjectId('5aba9fc532736325267ad2c4'),
 'level': 1,
 'title': 'Category:Biomedical engineering in fiction',
 'type': 'sub_category'}

In [13]:
db.pages.remove({})

F:\python\Lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


{'n': 6115, 'ok': 1.0}